In [ ]:
# %% OCR de kast.pdf (ruta relativa) + export e integración
!pip -q install pymupdf pillow pytesseract pandas pyarrow

import io, re
import fitz
from PIL import Image
import pytesseract
import pandas as pd
from pathlib import Path

# Fuerza el ejecutable si hace falta (ya lo tienes instalado)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
print("Tesseract:", pytesseract.get_tesseract_version())

# --- Ruta relativa por defecto, con fallback a tu ruta absoluta (con espacios) ---
PDF_PATH = Path("kast.pdf")
if not PDF_PATH.exists():
    PDF_PATH = Path(r"C:\Users\Rodrigo\Desktop\Rodrigo proyects\nlp-candidates\kast.pdf")
assert PDF_PATH.exists(), f"No existe: {PDF_PATH}"

def normalizar(x):
    if not x: return ""
    x = x.replace("\x00","")
    x = re.sub(r"[ \t]+"," ", x)
    x = re.sub(r"\s+\n","\n", x)
    x = re.sub(r"\n{3,}","\n\n", x)
    return x.strip()

def ocr_pymupdf(pdf_path, lang="spa", dpi=350, psm=6):
    doc = fitz.open(pdf_path)
    rows = []
    zoom = dpi / 72.0
    mat = fitz.Matrix(zoom, zoom)
    cfg = f"--oem 3 --psm {psm}"
    for i in range(len(doc)):
        pix = doc.load_page(i).get_pixmap(matrix=mat, alpha=False)
        img = Image.open(io.BytesIO(pix.tobytes("png")))
        try:
            txt = pytesseract.image_to_string(img, lang=lang, config=cfg) or ""
        finally:
            img.close()
        rows.append({"candidate":"kast","filename":PDF_PATH.name,"page":i+1,"text":normalizar(txt)})
    doc.close()
    return pd.DataFrame(rows)

# OCR principal; si detecta poco texto, reintenta con layout por columnas
pages_kast = ocr_pymupdf(PDF_PATH, lang="spa", dpi=350, psm=6)
if pages_kast["text"].str.len().sum() < 5000:
    pages_kast = ocr_pymupdf(PDF_PATH, lang="spa", dpi=400, psm=4)

# Métricas
pages_kast["n_chars"] = pages_kast["text"].str.len()
pages_kast["n_words"] = pages_kast["text"].str.split().apply(len)

# Documento agregado
docs_kast = (pages_kast
             .sort_values(["filename","page"])
             .groupby(["candidate","filename"], as_index=False)["text"]
             .apply(lambda s: "\n\n".join(s.values)))
docs_kast["n_chars"] = docs_kast["text"].str.len()
docs_kast["n_words"] = docs_kast["text"].str.split().apply(len)

# Persistencia (en el cwd)
pages_kast.to_parquet("kast_pages.parquet", index=False)
docs_kast.to_parquet("kast_document.parquet", index=False)
pages_kast.to_csv("kast_pages.csv", index=False, encoding="utf-8-sig")
docs_kast.to_csv("kast_document.csv", index=False, encoding="utf-8-sig")

print("Generados: kast_pages.{parquet,csv}, kast_document.{parquet,csv}")

# Integración opcional si existen tus datasets globales en el cwd
if Path("pages.parquet").exists() and Path("documents.parquet").exists():
    pages_all = pd.read_parquet("pages.parquet")
    docs_all  = pd.read_parquet("documents.parquet")
    pages_all = pd.concat([pages_all[pages_all.candidate!="kast"], pages_kast], ignore_index=True)
    docs_all  = pd.concat([docs_all[docs_all.candidate!="kast"],  docs_kast],  ignore_index=True)
    pages_all.to_parquet("pages.parquet", index=False)
    docs_all.to_parquet("documents.parquet", index=False)
    pages_all.to_csv("pages.csv", index=False, encoding="utf-8-sig")
    docs_all.to_csv("documents.csv", index=False, encoding="utf-8-sig")
    print("Corpus global actualizado: 'kast' reemplazado.")

print("\nResumen kast (n_words):")
print(docs_kast[["candidate","filename","n_words"]])
